## Cleaning the Excel file

In [1]:
import re
import pandas as pd
from datetime import date

file_name = "CCPA_Alojamentos.xlsx"

df = pd.read_excel(file_name)

/tmp/ipykernel_38222/2680388441.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df.drop("building", axis=1, inplace=True)
bedrooms = df.to_dict(orient='records')

## Putting the lodges dictionary into the bedroom model

In [3]:
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'r2e.settings')
django.setup()

from apps.center.models import Bedroom

bedrooms_with_problems = []
for bedroom in bedrooms:
   
    try:
        Bedroom.objects.create(**bedroom)
    except Exception as error:
        bedroom['error'] = str(error)
        bedrooms_with_problems.append(bedroom)

In [4]:
if len(bedrooms_with_problems) > 0:
    errors = pd.DataFrame(bedrooms_with_problems)
    errors.to_csv('bedrooms_with_problems.csv', index=False, encoding="utf-8")
    print(len(bedrooms_with_problems))
else:
    print("Everything is ok!")